In [1]:
DEVELOPMENT_MODE = False
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/jbloomAus/SAELens

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


C:\Users\huohu\AppData\Local\Temp\ipykernel_16656\740336833.py:16: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
C:\Users\huohu\AppData\Local\Temp\ipykernel_16656\740336833.py:17: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
import torch
if torch.cuda.is_available():
    device = "cuda"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"
torch.set_grad_enabled(False)

In [3]:
from jaxtyping import Float
from torch import Tensor
import torch
import torch.nn.functional as F
def get_cosine_similarity(
    dict_elements_1: Float[Tensor, "n_dense n_dict_1"],
    dict_elements_2: Float[Tensor, "n_dense n_dict_2"],
    p_norm: int = 2,
    dim: int = 1,
) -> Float[Tensor, "n_dict_1 n_dict_2"]:
    """Get the cosine similarity between the alive dictionary elements of two runs.

    Args:
        dict_elements_1: The alive dictionary elements of the first run.
        dict_elements_2: The alive dictionary elements of the second run.

    Returns:
        The cosine similarity between the alive dictionary elements of the two runs.
    """
    # Compute cosine similarity in pytorch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dict_elements_1 = dict_elements_1.to(device)
    dict_elements_2 = dict_elements_2.to(device)

    # Normalize the tensors
    dict_elements_1 = F.normalize(dict_elements_1, p=p_norm, dim=dim)
    dict_elements_2 = F.normalize(dict_elements_2, p=p_norm, dim=dim)

    # Compute cosine similarity using matrix multiplication
    cosine_sim: Float[Tensor, "n_dict_1 n_dict_2"] = torch.mm(dict_elements_1, dict_elements_2.T)
    return cosine_sim

In [4]:
import tqdm
from sae_lens import SAE
import numpy as np

hook_name_to_sae = []
length = 8013
d_sae = 24576
layers = 12
acts = torch.zeros((layers, length, d_sae)).to(device)
for layer in tqdm.tqdm(range(layers)):
    sae, cfg_dict, _ = SAE.from_pretrained(
        "gpt2-small-res-jb",
        f"blocks.{layer}.hook_resid_pre",
        device=device,
    )
    hook_name_to_sae.append(sae)
    with open(f"acts/activation0001_layer{layer}.npy", "rb") as f:
        for idx in range(length):
            acts[layer, idx] = torch.from_numpy(np.load(f)).to(device)

d:\code\sae\SAE_Geometry\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/12 [00:00<?, ?it/s]d:\code\sae\SAE_Geometry\.venv\Lib\site-packages\sae_lens\sae.py:145: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(
100%|██████████| 12/12 [00:52<00:00,  4.34s/it]


In [5]:
from sae_lens import HookedSAETransformer

model: HookedSAETransformer = HookedSAETransformer.from_pretrained("gpt2-small").to(
    device
)

from datasets import load_dataset

# Load the Skylion007/openwebtext dataset
dataset = load_dataset('Skylion007/openwebtext')
length = 8013
# Print the first example from the dataset
doc_length = torch.zeros([length]).to(device)

for idx in tqdm.tqdm(range(length)):
#for idx in tqdm.tqdm(range(10)):
    example = dataset['train'][idx]
    tokens = model.to_tokens([example['text']], prepend_bos=True)
    doc_length[idx] = tokens.shape[1]

weights = F.normalize(doc_length, p=1, dim=0)
expanded_weights = weights.unsqueeze(0).unsqueeze(2).expand(12, 8013, 24576)
acts_weighted = (acts * expanded_weights).sum(dim=1)
acts_weighted.shape

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


d:\code\sae\SAE_Geometry\.venv\Lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 8013/8013 [00:15<00:00, 510.55it/s]


torch.Size([12, 24576])

In [6]:
cos_sim = torch.zeros((layers, d_sae, d_sae))
for layer in tqdm.tqdm(range(12)):
    cos_sim[layer] = get_cosine_similarity(hook_name_to_sae[layer].W_dec, hook_name_to_sae[layer].W_dec).cpu()
    for length in range(d_sae):
        cos_sim[layer][length][length] = 0
cos_sim.shape

100%|██████████| 12/12 [00:07<00:00,  1.66it/s]


torch.Size([12, 24576, 24576])

In [7]:
indices = []
values = []
for layer in tqdm.tqdm(range(12)):
    value, index = cos_sim[layer].flatten().topk(100)
    indices.append(index)
    values.append(value)

100%|██████████| 12/12 [00:32<00:00,  2.71s/it]


In [8]:
freq_w_cos = []
for layer in tqdm.tqdm(range(12)):
    freq_layer = []
    for idx in range(100):
        freq_layer.append(indices[layer][idx]//24576)
        freq_layer.append(indices[layer][idx]%24576)
        freq = set(freq_layer)
        freq_w_cos.append(freq)

100%|██████████| 12/12 [00:00<00:00, 361.50it/s]


In [ ]:
freq_w_cos = []
for layer in tqdm.tqdm(range(12)):
    freq_layer = []
    for idx in range(5):
        freq_layer.append((indices[layer][idx]//24576).item())
        freq_layer.append((indices[layer][idx]%24576).item())
        freq = set(freq_layer)
    freq_w_cos.append(freq)

100%|██████████| 12/12 [00:00<00:00, 1403.36it/s]


In [89]:
import requests
with open('explanations_high_cos.json', 'w') as f:
    pbar = tqdm.tqdm(total=120)
    NEURONPEDIA_DOMAIN = "https://neuronpedia.org"
    model = "gpt2-small"
    dataset = "res-jb"
    for layer in range(12):
        for idx in range(10):
            index1 = (indices[layer][idx]//24576).item()
            index2 = (indices[layer][idx]%24576).item()           
            url = f"{NEURONPEDIA_DOMAIN}/api/feature/{model}/{layer}-{dataset}/{index2}"
            result = requests.get(url).json()
            if "explanations" in result and result["explanations"]:
                f.write(result["explanations"][0]["description"])
                f.write("\n")
            url = f"{NEURONPEDIA_DOMAIN}/api/feature/{model}/{layer}-{dataset}/{index1}"
            result = requests.get(url).json()
            if "explanations" in result and result["explanations"]:
                f.write(result["explanations"][0]["description"])
                f.write("\n")
            pbar.update(1)

62it [00:43,  1.42it/s][00:00<?, ?it/s]
100%|██████████| 120/120 [01:18<00:00,  1.48it/s]

In [41]:
import plotly.express as px
import pandas as pd
import plotly.colors as pc

stats_dfs = []
colors = pc.n_colors("rgb(5, 200, 200)", "rgb(200, 10, 10)", 13, colortype="rgb")
for layer in tqdm.tqdm(range(12)):
    cos_sim_df = pd.DataFrame({"cos": acts_weighted[layer][list(freq_w_cos[layer])].cpu().numpy(), "layer": layer})
    stats_dfs.append(cos_sim_df)
cos_df = pd.concat(stats_dfs, axis=0)

# TODO: how can I improve this plot?
# RESULT1: Cos sim by layers
fig = px.box(
    cos_df,
    x="layer",
    y = "cos",
    width=1200,
    height=600,
    color="layer",
    color_discrete_sequence=colors,
    title=f"cosine similarity in the same layer in GPT2-Small-Res-JB",
    labels={"layer": "Layer", "cos": "cos"},
)
fig.update_xaxes(showticklabels=True, dtick=1)

# increase font size
fig.update_layout(font=dict(size=16))
fig.show()

100%|██████████| 12/12 [00:00<00:00, 4000.93it/s]


In [42]:
indices = []
values = []
for layer in tqdm.tqdm(range(12)):
    value, index = cos_sim[layer].flatten().topk(100, largest=False)
    indices.append(index)
    values.append(value)

100%|██████████| 12/12 [00:37<00:00,  3.15s/it]


In [43]:
freq_w_cos = []
for layer in tqdm.tqdm(range(12)):
    freq_layer = []
    for idx in range(100):
        freq_layer.append((indices[layer][idx]//24576).item())
        freq_layer.append((indices[layer][idx]%24576).item())
        freq = set(freq_layer)
    freq_w_cos.append(freq)
import plotly.express as px
import pandas as pd
import plotly.colors as pc

stats_dfs = []
colors = pc.n_colors("rgb(5, 200, 200)", "rgb(200, 10, 10)", 13, colortype="rgb")
for layer in tqdm.tqdm(range(12)):
    cos_sim_df = pd.DataFrame({"cos": acts_weighted[layer][list(freq_w_cos[layer])].cpu().numpy(), "layer": layer})
    stats_dfs.append(cos_sim_df)
cos_df = pd.concat(stats_dfs, axis=0)

# TODO: how can I improve this plot?
# RESULT1: Cos sim by layers
fig = px.box(
    cos_df,
    x="layer",
    y = "cos",
    width=1200,
    height=600,
    color="layer",
    color_discrete_sequence=colors,
    title=f"cosine similarity in the same layer in GPT2-Small-Res-JB",
    labels={"layer": "Layer", "cos": "cos"},
)
fig.update_xaxes(showticklabels=True, dtick=1)

# increase font size
fig.update_layout(font=dict(size=16))
fig.show()

100%|██████████| 12/12 [00:00<00:00, 2659.67it/s]


In [45]:
indecs_max = []
indices = []
values = []
values_max = []
for layer in tqdm.tqdm(range(12)):
    value, index = acts_weighted[layer].topk(100)
    value_max, index_max = acts[layer].max(dim=1)[0].topk(100)
    indices.append(index)
    values.append(value)
    indecs_max.append(index_max)
    values_max.append(value_max)

100%|██████████| 12/12 [00:00<00:00, 748.55it/s]


In [49]:
indices[0]

tensor([ 9337, 13908,  4172, 12555, 14015,  8687,  6167,  4027, 12167, 14009,
         7632,   745, 22148,  4274, 23825, 15145,   556, 18851, 11633,   182,
        21611, 18069, 18877, 15186, 19792, 18178, 12839,  8722, 22543, 16410,
         3816,  6686,  7474, 23733,   363, 21616, 18336,  3508,  2218, 20750,
         2945, 15684,  2718, 15205,  2448, 19595, 19430,  8472,  5748,  9440,
          450,  6530, 18597,  6911,  3219, 22518, 16934,  2213, 17593,  6635,
        15430, 20824,  1169,  7795,  1934,  2924, 11787, 12204,  9758,  3382,
        18861,  6715, 15082, 18458, 21686, 19489, 16852, 16627, 16491, 14697,
        15286,  3336,  6522, 10358, 21061,  2326, 12047, 12687, 23116,  9462,
        10633,  4942, 11641, 20246, 16149, 14687, 14149,  8317, 23823, 24038],
       device='cuda:0')

In [82]:
import requests
import json
from typing import Any
def get_neuronpedia_feature(
    feature: int,  f, layer: int, model: str = "gpt2-small", dataset: str = "res-jb",
) -> dict[str, Any]:
    """Fetch a feature from Neuronpedia API."""
    NEURONPEDIA_DOMAIN = "https://neuronpedia.org"
    url = f"{NEURONPEDIA_DOMAIN}/api/feature/{model}/{layer}-{dataset}/{feature}"
    result = requests.get(url).json()
    if "explanations" in result and result["explanations"]:
        f.write(result["explanations"][0]["description"])
    else:
        return None
    

In [85]:
with open('explanations.json', 'w') as f:
    pbar = tqdm.tqdm(total=1200)
    NEURONPEDIA_DOMAIN = "https://neuronpedia.org"
    model = "gpt2-small"
    dataset = "res-jb"
    for layer in range(12):
        for idx in range(100):
            feature=indices[layer][idx].item()
            url = f"{NEURONPEDIA_DOMAIN}/api/feature/{model}/{layer}-{dataset}/{feature}"
            result = requests.get(url).json()
            if "explanations" in result and result["explanations"]:
                f.write(result["explanations"][0]["description"])
                f.write("\n")
                print(result["explanations"][0]["description"])
            pbar.update(1)

  1%|▏         | 16/1200 [00:05<05:50,  3.38it/s]

terms related to environmental issues, specifically related to overfishing and its consequences


  2%|▏         | 18/1200 [00:05<05:39,  3.48it/s]

references to creatures or beings described as "globsters" or similar terms


  2%|▏         | 26/1200 [00:08<06:10,  3.17it/s]

...

 38%|███▊      | 451/1200 [02:35<04:08,  3.01it/s]

special characters or symbols


 38%|███▊      | 452/1200 [02:36<04:07,  3.02it/s]

instances of words related to confidence and self-belief


UnicodeEncodeError: 'charmap' codec can't encode character '\u0122' in position 85: character maps to <undefined>

In [93]:
indices = []
values = []
for layer in tqdm.tqdm(range(12)):
    value, index = acts_weighted[layer].topk(100, largest=False)
    indices.append(index)
    values.append(value)

with open('explanations.json', 'w') as f:
    pbar = tqdm.tqdm(total=1200)
    NEURONPEDIA_DOMAIN = "https://neuronpedia.org"
    model = "gpt2-small"
    dataset = "res-jb"
    for layer in range(12):
        for idx in range(100):
            feature=indices[layer][idx].item()
            url = f"{NEURONPEDIA_DOMAIN}/api/feature/{model}/{layer}-{dataset}/{feature}"
            result = requests.get(url).json()
            if "explanations" in result and result["explanations"]:
                f.write(result["explanations"][0]["description"])
                f.write("\n")
            pbar.update(1)

 43%|████▎     | 517/1200 [02:35<04:06,  2.77it/s]

UnicodeEncodeError: 'charmap' codec can't encode character '\u010a' in position 97: character maps to <undefined>